In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import numpy as np
import pandas as pd
import tensorflow as tf

import dataloader as DL
import layers as CLayers
import train_utils as TU

from sklearn.model_selection import train_test_split

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

## Load data

In [ ]:
df = pd.read_csv('./data/ADNI_saliency.csv', low_memory=False)
date = '20240118_2300'
input_name = 'AbsDiff'
timepoint = '4Y'
df = df[df[f'CONV_STATE_{timepoint}'] != -1]
unique_rids = df['RID'].unique()

In [ ]:
train_rids, valid_rids = train_test_split(unique_rids, test_size=0.2, random_state=2024)

In [ ]:
with open(f'./data/train_rids_{timepoint}.txt', 'w') as file:
    for rid in train_rids:
        file.write(f"{rid}\n")
with open(f'./data/val_rids_{timepoint}.txt', 'w') as file:
    for rid in valid_rids:
        file.write(f"{rid}\n")

In [ ]:
with open(f'./data/train_rids_{timepoint}.txt', 'r') as file:
    train_rids = [line.strip() for line in file]
train_rids = [int(float(rid)) for rid in train_rids]
# Load validation RIDs from the text file
with open(f'./data/val_rids_{timepoint}.txt', 'r') as file:
    valid_rids = [line.strip() for line in file]
valid_rids = [int(float(rid)) for rid in valid_rids]

In [ ]:
data_file = './data/ADNI_saliency.csv'
batch_size  = 30
num_classes = 3
timesteps   = int(timepoint[0])*2+1
image_shape = (84,48,42)
use_pe = False
train_ds = DL.InputFunction(filepath    = data_file,
                            list_rids   = train_rids,
                            labeltime   = timepoint,
                            num_classes = num_classes,
                            image_shape = image_shape,
                            use_pe      = use_pe,
                            timesteps   = timesteps, 
                            batch_size  = batch_size,
                            drop_last=True,
                            shuffle=True)
valid_ds = DL.InputFunction(filepath    = data_file,
                            list_rids   = valid_rids,
                            labeltime   = timepoint,
                            num_classes = num_classes,
                            image_shape = image_shape,
                            use_pe      = use_pe,
                            timesteps   = timesteps,
                            batch_size  = batch_size,
                            shuffle=True)

In [ ]:
train_steps = train_ds.steps_per_epoch()
if (not train_ds.drop_last) and (train_ds.steps_per_epoch()*batch_size<train_ds.size()):
    train_steps += 1

In [ ]:
# max_time = 0
# for i, (feat, label) in enumerate(valid_ds()):
#     temp_time = feat['deltas'].shape[1]
#     if temp_time > max_time:
#         max_time = temp_time
# print(max_time)

In [ ]:
# for i, (feat, label) in enumerate(train_ds()):
#     if i==1:
#         break

In [ ]:
# feat['AbsDiff'].numpy().max()

## Create model

#### Define unimodel

##### #1

In [ ]:
# model_config = dict(
#     input_name=input_name,
#     num_classes=num_classes,
#     timesteps=timesteps,
#     scn_channels=[16, 32, 64],
#     aspp_channels=timesteps,
#     tcn_channels=[64, 32],
#     fc_units=[256, 128],
#     dilation_rates=[6,12,18],
#     kernel_size=3,
#     pool_size=2,
#     dropout=0.35,
# )

In [ ]:
# uni_model = CLayers.UnimodelS1(**model_config)

##### #2

In [ ]:
model_config = dict(
    input_name=input_name,
    num_classes=num_classes,
    num_filters=[8,16,16,32,32],
    fc_units=[64, 32],
    kernel_size=3,
    pool_size=2,
    dropout=0.4,
)

In [ ]:
uni_model = CLayers.UnimodelS1_CNN(**model_config)

## Create trainer

In [ ]:
learning_rate = 1e-3
num_epochs    = 50
optimizer     = tf.keras.optimizers.Adam(learning_rate=learning_rate, weight_decay=1e-5)

In [ ]:
trainer = TU.TrainAndEvaluateS1(
    model=uni_model,
    model_dir=f"./checkpoints/unimodel_{input_name}_{date}_{timepoint}",
    train_dataset=train_ds(),
    eval_dataset=valid_ds(),
    num_epochs=num_epochs,
    train_steps=train_steps,
    optimizer=optimizer
)

In [ ]:
trainer.train_and_evaluate()

In [ ]:
trainer.save_model('save_model')

## Prediction

In [ ]:
import predict_utils as PU

In [ ]:
predictor = PU.Predictor(trainer.model, trainer.model_dir, input_name, ckpt_name='save_model')

In [ ]:
predictor.predict(valid_ds)

In [ ]:
results = predictor.get_performance()

In [ ]:
results

In [ ]:
predictor.plot_confusion_matrix(columns=['Non-Converted', 'Converted', 'AD'], save_path=f'figures/unimodel_{input_name}_{date}_{timepoint}.png')